这是一个二维的流体力学模拟仿真程序（插入必要的库）

In [1]:
import matplotlib.pyplot as plt  
import numpy as np  
from tqdm import tqdm  #进度条库
import os

设置运行条件

In [2]:
N_POINTSX = 50  #X轴线上的网格点数
N_POINTSY = 50  #Y轴线上的网格点数
DOMAIN_SIZEX = 1.0  #模拟区域X大小
DOMAIN_SIZEY = 1.0  #模拟区域Y大小
N_ITERATIONS = 500  #迭代次数  
TIME_STEP_LENGTH = 0.001  #时间步长
KINEMATIC_VISCOSITY = 0.1  #运动粘度 
DENSITY = 1.0  #密度

N_PRESSURE_POISSON_ITERATIONS = 50  #Poisson迭代的次数  

设置场以及场中的矢量与标量

In [3]:
def initialize_fields():  
    element_length = DOMAIN_SIZEX / (N_POINTSX - 1)
    x = np.linspace(0.0, DOMAIN_SIZEX, N_POINTSX)  
    y = np.linspace(0.0, DOMAIN_SIZEY, N_POINTSY)  
    X, Y = np.meshgrid(x, y)  
    u = np.zeros((N_POINTSX, N_POINTSY))  
    v = np.zeros((N_POINTSX, N_POINTSY))  
    p = np.zeros((N_POINTSX, N_POINTSY))  
      
    return X, Y, u, v, p, element_length  #element网格间距

中心差分计算

In [4]:
def central_difference(f, axis, element_length):    
    diff = np.zeros_like(f)    
    if axis == 'x':    
        diff[1:-1, 1:-1] = (f[1:-1, 2:] - f[1:-1, 0:-2]) / (2 * element_length)    
    elif axis == 'y':    
        diff[1:-1, 1:-1] = (f[2:, 1:-1] - f[0:-2, 1:-1]) / (2 * element_length)    
    return diff 

计算拉普拉斯算子

In [5]:
def laplace(f, element_length):    
    diff = np.zeros_like(f)    
    diff[1:-1, 1:-1] = (f[1:-1, 0:-2] + f[0:-2, 1:-1] - 4 * f[1:-1, 1:-1] + f[1:-1, 2:] + f[2:, 1:-1]) / (element_length**2)    
    return diff  

设置边界条件

In [6]:
def set_boundary_conditions(u, v, p):  
    #u=水平速度 v=垂直速度 p=压力
    #u[0, :] = 0  #底部水平速度设置
    u[:, 0] = 10.0  #左侧水平速度设置
    u[:, -1] = -10.0 #右侧水平速度设置 
    #u[-1, :] = -10#顶部水平速度设置  
    v[0, :] = 10.0  #底部垂直速度设置
    #v[:, 0] = -10  #左侧垂直速度设置
    #v[:, -1] = 10 #右侧垂直速度设置 
    v[-1, :] = -10.0 #底部垂直速度设置
    #u[25,0] = 10.0 #设置中间的水平速度为0
    #u[25,-1] = -10.0 #设置中间的水平速度为0
    #v[25,:] = 0.0 #设置中间的垂直速度为0

    p[:, -1] = p[:, -2]  
    p[0, :] = p[1, :]  
    p[:, 0] = p[:, 1]  
    p[-1, :] = 0.0  

主程序

In [7]:
def main_for_a_point():     
    X, Y, u_prev, v_prev, p_prev, element_length = initialize_fields()    
    
    for _ in tqdm(range(N_ITERATIONS)):    
        d_u_prev__d_x = central_difference(u_prev, 'x', element_length)    
        d_u_prev__d_y = central_difference(u_prev, 'y', element_length)    
        d_v_prev__d_x = central_difference(v_prev, 'x', element_length)    
        d_v_prev__d_y = central_difference(v_prev, 'y', element_length)    
        laplace__u_prev = laplace(u_prev, element_length)    
        laplace__v_prev = laplace(v_prev, element_length)    
  
        u_tent = u_prev + TIME_STEP_LENGTH * (-(u_prev * d_u_prev__d_x + v_prev * d_u_prev__d_y) + KINEMATIC_VISCOSITY * laplace__u_prev)  
        v_tent = v_prev + TIME_STEP_LENGTH * (-(u_prev * d_v_prev__d_x + v_prev * d_v_prev__d_y) + KINEMATIC_VISCOSITY * laplace__v_prev)
        set_boundary_conditions(u_tent, v_tent, p_prev)  
  
        d_u_tent__d_x = central_difference(u_tent, 'x',  element_length)  
        d_v_tent__d_y = central_difference(v_tent, 'y',  element_length)  
  
        rhs = (DENSITY / TIME_STEP_LENGTH) * (d_u_tent__d_x + d_v_tent__d_y)  
  
        for _ in range(N_PRESSURE_POISSON_ITERATIONS):  
            p_next = np.zeros_like(p_prev)  
            p_next[1:-1, 1:-1] = 0.25 * (p_prev[1:-1, 0:-2] + p_prev[0:-2, 1:-1] + p_prev[1:-1, 2:] + p_prev[2:, 1:-1] - element_length**2 * rhs[1:-1, 1:-1])  
  
            p_next[:, -1] = p_next[:, -2]  
            p_next[0, :] = p_next[1, :]  
            p_next[:, 0] = p_next[:, 1]  
            p_next[-1, :] = 0.0  
  
            p_prev = p_next  
  
        d_p_next__d_x = central_difference(p_next, 'x', element_length)  
        d_p_next__d_y = central_difference(p_next, 'y',  element_length)  
  
        u_next = u_tent - TIME_STEP_LENGTH / DENSITY * d_p_next__d_x  
        v_next = v_tent - TIME_STEP_LENGTH / DENSITY * d_p_next__d_y  
  
        set_boundary_conditions(u_next, v_next, p_next)  
  
        u_prev = u_next  
        v_prev = v_next  
        p_prev = p_next  
    
    plt.figure()  
    #plt.contour(X, Y, p_next)  
    #plt.colorbar()  
    plt.quiver(X, Y, u_next, v_next, color="black", scale=70, width=0.001)  
    save_path = 'my_plot.png'  
    plt.savefig(save_path) 
 
    plt.close() #显示图像换成plt.show()

运行多次主函数，模拟一段时间的流体

In [8]:
def main_for_period():     
    X, Y, u_prev, v_prev, p_prev, element_length = initialize_fields()
    folder_name = 'new_folder'# 定义文件夹的名称
    os.makedirs(folder_name, exist_ok=True)# 使用 os.makedirs 创建文件夹,exist_ok=True 参数表示如果文件夹已存在，则不抛出错误
    
    
    for i in range(N_ITERATIONS):    
        d_u_prev__d_x = central_difference(u_prev, 'x', element_length)    
        d_u_prev__d_y = central_difference(u_prev, 'y', element_length)    
        d_v_prev__d_x = central_difference(v_prev, 'x', element_length)    
        d_v_prev__d_y = central_difference(v_prev, 'y', element_length)    
        laplace__u_prev = laplace(u_prev, element_length)    
        laplace__v_prev = laplace(v_prev, element_length)    
  
        u_tent = u_prev + TIME_STEP_LENGTH * (-(u_prev * d_u_prev__d_x + v_prev * d_u_prev__d_y) + KINEMATIC_VISCOSITY * laplace__u_prev)  
        v_tent = v_prev + TIME_STEP_LENGTH * (-(u_prev * d_v_prev__d_x + v_prev * d_v_prev__d_y) + KINEMATIC_VISCOSITY * laplace__v_prev)
        set_boundary_conditions(u_tent, v_tent, p_prev)  
  
        d_u_tent__d_x = central_difference(u_tent, 'x',  element_length)  
        d_v_tent__d_y = central_difference(v_tent, 'y',  element_length)  
  
        rhs = (DENSITY / TIME_STEP_LENGTH) * (d_u_tent__d_x + d_v_tent__d_y)  
  
        for _ in range(N_PRESSURE_POISSON_ITERATIONS):  
            p_next = np.zeros_like(p_prev)  
            p_next[1:-1, 1:-1] = 0.25 * (p_prev[1:-1, 0:-2] + p_prev[0:-2, 1:-1] + p_prev[1:-1, 2:] + p_prev[2:, 1:-1] - element_length**2 * rhs[1:-1, 1:-1])  
  
            p_next[:, -1] = p_next[:, -2]  
            p_next[0, :] = p_next[1, :]  
            p_next[:, 0] = p_next[:, 1]  
            p_next[-1, :] = 0.0  
  
            p_prev = p_next  
  
        d_p_next__d_x = central_difference(p_next, 'x', element_length)  
        d_p_next__d_y = central_difference(p_next, 'y',  element_length)  
  
        u_next = u_tent - TIME_STEP_LENGTH / DENSITY * d_p_next__d_x  
        v_next = v_tent - TIME_STEP_LENGTH / DENSITY * d_p_next__d_y  
  
        set_boundary_conditions(u_next, v_next, p_next)  
  
        u_prev = u_next  
        v_prev = v_next  
        p_prev = p_next
        Z = np.sqrt(u_next**2 + v_next**2)  

        plt.figure()  
        #plt.contour(X, Y, p_next)  
        #plt.colorbar()
        plt.contourf(X,Y,Z,100,cmap=plt.cm.get_cmap("rainbow"))  
        plt.quiver(X, Y, u_next, v_next, color="black", scale=70, width=0.001)
        plt.streamplot(X, Y, u_next, v_next, density=1, color='k',linewidth=0.5)
        save_path = 'new_folder/my_plot' + str(i) + '.png'  
        plt.savefig(save_path)
        plt.close() #显示图像换成plt.show()


运行

In [9]:
main_for_period()

/tmp/ipykernel_8814/2829841226.py:51: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  plt.contourf(X,Y,Z,100,cmap=plt.cm.get_cmap("rainbow"))


画出动图

In [14]:
import imageio

def compose_mp4(N_ITERATIONS):
    mp4_images = []
    for i in range(0,N_ITERATIONS):
        mp4_images.append(imageio.imread('new_folder/my_plot' + str(i) + '.png'))
    imageio.mimsave("test.mp4", mp4_images, format='mp4', fps=10)  

compose_mp4(N_ITERATIONS)


/tmp/ipykernel_8814/436048640.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mp4_images.append(imageio.imread('new_folder/my_plot' + str(i) + '.png'))
